In [1]:
rm(list = ls())
path.data.raw <- "./Data/GSE/RAW/"
path.data.filtred <- "./Data/GSE/Filtred/"
path.data.out <- "./Data/Output/"

In [ ]:
if (!require("Seurat")) {
    install.packages("Seurat")
    library(Seurat)
}
if (!require("hdf5r")) { 
    install.packages("hdf5r")
    library(hdf5r)
}

Ładowanie wymaganego pakietu: Seurat

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“nie ma pakietu o nazwie ‘Seurat’”
instalowanie dodatkowych zależności ‘sys’, ‘bitops’, ‘stringi’, ‘fs’, ‘codetools’, ‘askpass’, ‘yaml’, ‘tidyselect’, ‘cpp11’, ‘generics’, ‘rprojroot’, ‘gtools’, ‘caTools’, ‘colorspace’, ‘stringr’, ‘sass’, ‘jquerylib’, ‘BH’, ‘sitmo’, ‘gtable’, ‘survival’, ‘globals’, ‘listenv’, ‘parallelly’, ‘isoband’, ‘mgcv’, ‘withr’, ‘plyr’, ‘curl’, ‘mime’, ‘openssl’, ‘R6’, ‘magrittr’, ‘pkgconfig’, ‘zoo’, ‘lattice’, ‘viridisLite’, ‘htmlwidgets’, ‘tidyr’, ‘dplyr’, ‘lazyeval’, ‘crosstalk’, ‘purrr’, ‘data.table’, ‘promises’, ‘RcppTOML’, ‘here’, ‘rappdirs’, ‘gplots’, ‘farver’, ‘labeling’, ‘munsell’, ‘reshape2’, ‘gridExtra’, ‘RcppArmadillo’, ‘httpuv’, ‘xtable’, ‘fontawesome’, ‘sourcetools’, ‘later’, ‘commonmark’, ‘bslib’, ‘cachem’, ‘spatstat.data’, ‘nlme’, ‘rpart’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘abind’, ‘tensor’, ‘goftest’, ‘del

In [ ]:
path.data = path.data.raw

h5_files <- list.files(path.data, pattern = "*.h5")
h5_files <- mapply(paste, path.data, h5_files, sep='')

file = h5_files[1]

pbmc.data <- Read10X_h5(file)
pbmc <- CreateSeuratObject(pbmc.data)
rm(pbmc.data)

In [ ]:
pbmc[["percent.mt"]] <- PercentageFeatureSet(pbmc, pattern = "^MT-")
pbmc[["percent.mt"]]

# Visualize QC metrics as a violin plot

In [ ]:
VlnPlot(pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, pt.size = 0)

FeatureScatter is typically used to visualize feature-feature relationships, but can be used for anything calculated by the object, i.e. columns in object metadata, PC scores etc.

In [ ]:
plot1 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

In [ ]:
pbmc <- subset(pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)

pbmc <- NormalizeData(pbmc, normalization.method = "LogNormalize", scale.factor = 10000)

pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes

In [ ]:
top10 <- head(VariableFeatures(pbmc), 10)

Plot variable features with and without labels to identify the 10 most highly variable genes

In [ ]:
plot1 <- VariableFeaturePlot(pbmc)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot1 + plot2

In [ ]:
saveRDS(pbmc, file=paste(path.data.out, '2-cleaned.rds', sep=''))

In [ ]:
pbmc <- readRDS(file=paste(path.data.out, '2-cleaned.rds', sep=''))

all.genes <- rownames(pbmc)
pbmc <- ScaleData(pbmc, features = all.genes)

pbmc <- RunPCA(pbmc, features = VariableFeatures(object = pbmc))

In [ ]:
print(pbmc[["pca"]], dims = 1:5, nfeatures = 5)

In [ ]:
VizDimLoadings(pbmc, dims = 1:2, reduction = "pca")

In [ ]:
DimPlot(pbmc, reduction = "pca")

In [ ]:
DimHeatmap(pbmc, dims = 1, cells = 500, balanced = TRUE)

In [ ]:
DimHeatmap(pbmc, dims = 1:15, cells = 500, balanced = TRUE)

In [ ]:
pbmc <- JackStraw(pbmc, num.replicate = 100)
pbmc <- ScoreJackStraw(pbmc, dims = 1:20)

In [ ]:
JackStrawPlot(pbmc, dims = 1:15)

In [ ]:
pbmc <- FindNeighbors(pbmc, dims = 1:10)
pbmc <- FindClusters(pbmc, resolution = 0.5)

In [ ]:
head(Idents(pbmc), 5)

In [ ]:
pbmc <- RunUMAP(pbmc, dims = 1:10)

In [ ]:
DimPlot(pbmc, reduction = "umap")

In [ ]:
saveRDS(pbmc, file = paste(path.data.out, "3-scaled-pca.rds", SEP=""))

In [ ]:
cluster2.markers <- FindMarkers(pbmc, 
                                ident.1 = 2, 
                                min.pct = 0.25)
head(cluster2.markers, n = 5)

In [ ]:
cluster5.markers <- FindMarkers(pbmc, 
                                ident.1 = 5, 
                                ident.2 = c(0, 3), 
                                min.pct = 0.25)
head(cluster5.markers, n = 5)

In [ ]:
pbmc.markers <- FindAllMarkers(pbmc, 
                               only.pos = TRUE, 
                               min.pct = 0.25, 
                               logfc.threshold = 0.25)
pbmc.markers %>%
    group_by(cluster) %>%
    slice_max(n = 2, order_by = avg_log2FC)

In [ ]:
cluster0.markers <- FindMarkers(pbmc, 
                                ident.1 = 0, 
                                logfc.threshold = 0.25, 
                                test.use = "roc", 
                                only.pos = TRUE)

In [ ]:
print(pbmc[["pca"]], dims = 1:5, nfeatures = 5)

In [ ]:
VlnPlot(pbmc, features = c("SERPINA1", "CTSH"))

In [ ]:
VlnPlot(pbmc, features = c("SPARCL1", "CLDN5"), slot = "counts", log = TRUE)

In [ ]:
FeaturePlot(pbmc, features = c(
  "SERPINA1", "CTSH", "APOC1", "NPC2"
))

In [ ]:
pbmc.markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10

In [ ]:
DoHeatmap(pbmc, features = top10$gene) + NoLegend()